In [1]:
#for dev purpose
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [2]:
# import python packages
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta
from snp_query_box import DsnpHelperFunction, snpQueries
from snp_query_box.dsnp_transform import dsnpTransform

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating ked_gap_status_df in the shared storage")

User: a845514 Started creating ked_gap_status_df in the shared storage


In [3]:
# set the starting time
start_time = time.time()

In [4]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)
first_date_of_month = dt.datetime.today().replace(day=1)
first_date_of_last_month = first_date_of_month - relativedelta(months=1)
call_log_date2 = pd.to_datetime(first_date_of_last_month).date()
print(call_log_date2)
call_log_date1 = pd.to_datetime(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1)).date()
print(call_log_date1)

reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

2025-05-01
2024-06-01
2025-05-31


In [5]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files/2025-05-31
The folder already exist
Output files will sit in the folder above.


In [6]:
#locate data dump folder
if new_pull:
    pull_ked_gap_status = snpQueries.pull_ked_gap_status(reporting_year=2025)
    ked_gap_status_df = dsnpTransform.get_ked_gap_status(pull_ked_gap_status)
    ked_gap_status_df.to_parquet(f"{output_path}/ked_gap_status_df.parquet")
else:
    try:
        ked_gap_status_df = pd.read_parquet(f"{output_path}/ked_gap_status_df.parquet")

    except:
        pull_ked_gap_status = snpQueries.pull_ked_gap_status(reporting_year=2025)
        ked_gap_status_df = dsnpTransform.get_ked_gap_status(pull_ked_gap_status)

        ked_gap_status_df.to_parquet(f"{output_path}/ked_gap_status_df.parquet")

100%|██████████| 1036411/1036411 [00:20<00:00, 50635.73it/s]


In [7]:
print(ked_gap_status_df.shape)
print(ked_gap_status_df.head(3))

(674819, 8)
       SourceMemberID  StarsMemberID           MBI  MeasureID   
352088   101609903700       12508963   1A00F91YE22     600484  \
82701    101194058700         999472  1A00GE2YA30      600484   
452065   101833090400       22432653   1A00GN8QH45     600484   

       LogicalOperatorCode  ReportingYear             LastUpdated   
352088                  OR           2025 2025-05-12 19:33:25.000  \
82701                   OR           2025 2025-05-24 11:30:50.447   
452065                  OR           2025 2025-05-12 19:33:25.000   

           ked_status_desc  
352088  [yes:eGFR no:uACR]  
82701    [no:eGFR no:uACR]  
452065  [yes:eGFR no:uACR]  


In [10]:
ked_gap_status_df["ked_status_desc"].value_counts()

ked_status_desc
[yes:eGFR no:uACR]                               365582
[no:eGFR no:uACR]                                245777
[no:eGFR yes:uACR, urine test > 4 days apart]     43762
[no:eGFR yes:uACR]                                19017
[urine test > 4 days apart]                         681
Name: count, dtype: int64

In [9]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))

Process time -- 77.02029848098755 seconds
